# This code to generate answer column for Hallubench dataset using Simple RAG pipeline with Mistral locally using Langchain Ollama

In [1]:
#@title { vertical-output: true}
# Install ollama: The Ollama API is now available at 127.0.0.1:11434
! curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
#@title { vertical-output: true}
import subprocess
process = subprocess.Popen("ollama serve", shell=True)
# credit: https://stackoverflow.com/questions/78437376/run-ollama-run-llama3-in-colab-raise-err-error-could-not-connect-to-ollama

# download the Mistral model
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏  10 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   1% ▕▏  43 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕▏  63 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 104 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 119 MB/4.1 GB    

In [4]:
#@title { vertical-output: true}
!pip install -q langchain
!pip install -U -q langchain-community
!pip install -U langchain-huggingface sentence-transformers
!pip install -q chromadb
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67

In [5]:
#@title { vertical-output: true}
import pandas as pd
from datasets import Dataset, Features, Sequence, Value
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.retrievers import MultiQueryRetriever
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from tqdm import tqdm

# Initialize Ollama LLM and Embeddings
llm: BaseLanguageModel = Ollama(model="mistral", temperature = 0.1)

In [6]:
#@title { vertical-output: true}
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

<ipython-input-6-f0e2007b65f4>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
#@title { vertical-output: true}
# Load the dataset from a local CSV file

df = pd.read_csv('/content/halubench_short.csv')

# Ensure the dataset has the required columns: 'question', 'contexts', 'ground_truth'
required_columns = ['question', 'contexts', 'ground_truth']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"The dataset must contain these columns: {required_columns}")


In [8]:
#@title { vertical-output: true}
def calculation(row, llm=llm, embed_model=embed_model):
    # Split the context into chunks
    context = str(row['contexts'])
    question = str(row['question'])

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=16)
    all_splits = text_splitter.split_text(context)

    # Create a vectorstore from the splits
    vectorstore = Chroma.from_texts(texts=all_splits, embedding=embed_model)

    # Define the query prompt for MultiQueryRetriever
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI language model assistant. Your task is to generate 3
        different versions of the given user question to retrieve relevant documents from
        a vector database. Make the questions simple and concise. Provide these alternative questions separated by newlines.
        Original question: {question}""",
    )

    # Create MultiQueryRetriever
    retriever = MultiQueryRetriever.from_llm(
        vectorstore.as_retriever(search_kwargs={"k": 3}), llm, prompt=QUERY_PROMPT
    )

    # Retrieve documents
    retriever_results = retriever.get_relevant_documents(question)

    # Define the prompt template for the final answer
    template = """
    You are a helpful AI assistant. Use the following pieces of information to answer the user's question. Provide a concise answer in one sentence only. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context: {context}
    Question: {question}
    Only provide the answer from the context, nothing else.
    """

    prompt = ChatPromptTemplate.from_template(template)

    # Create the RAG pipeline
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # Invoke the chain with the question
    result = chain.invoke(question)

    return {
        'answer': result,
        'retrieved_docs': [doc.page_content for doc in retriever_results]
    }

# Apply the calculation function to each row with a progress bar
tqdm.pandas(desc="Processing rows")
results = df.progress_apply(calculation, axis=1)

# Add new columns to the dataframe
df['answer'] = results.apply(lambda x: x['answer'])
df['retrieved_docs'] = results.apply(lambda x: x['retrieved_docs'])

print("Processing complete!")

Processing rows:   0%|          | 0/60 [00:00<?, ?it/s]<ipython-input-8-4eb7c3cc60d7>:28: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever_results = retriever.get_relevant_documents(question)
Processing rows: 100%|██████████| 60/60 [05:58<00:00,  5.97s/it]

Processing complete!


In [11]:
# Select required columns for the Hugging Face dataset
df_hf = df[required_columns + ['answer', 'retrieved_docs']]

# Define the schema for the Hugging Face dataset
features = Features({
    'question': Value('string'),
    'contexts': Sequence(Value('string')),
    'ground_truth': Value('string'),
    'answer': Value('string'),
    'retrieved_docs': Sequence(Value('string'))
})

In [12]:
# Save the DataFrame with generated answers to a CSV file
output_csv_path = 'halubench_short_answers_MultiQuery_Rag_langchain_ollama_3.csv'
df.to_csv(output_csv_path, index=False)